In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from minepy import MINE

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#drop相同列
def drop_col(data):
    data = data.fillna(data.mean())      
    for line in data.columns:
        if len(data[line].unique()) <= 3:
            data = data.drop([line], axis=1)    
    return data

In [3]:
#四分位数处理异常值
def quantile_dropout(data):  
    for c in data.columns:
        Q1 = data[c].quantile(q=0.25, interpolation='linear')
        Q3 = data[c].quantile(q=0.75, interpolation='linear')   
    
        min_v = Q1 - 1.5 * (Q3 - Q1)
        max_v = Q3 + 1.5 * (Q3 - Q1)
    
        data[c][(data[c] >= max_v) | (data[c] <= min_v)] = data[c].median()
           
    return data

In [4]:
#标准化
def scala(data):
 #   for col in data.columns:            
 #       data[col] = (data[col] - data[col].mean()) / data[col].std(ddof=0) 
 #   data = data.fillna(0)   
    data = scale(data)
    return data

In [5]:
def date_cols(data):
    for col in data:
        if data[col].min() > 1e13:
            data = data.drop([col], axis=1)   
    return data

In [6]:
def evalerror(y, y_pred):
    loss = np.sum(np.square(y - y_pred))
    n = len(y)
    return loss / n

In [7]:
data_set = pd.read_csv('train/input_data.csv')

In [8]:
y = pd.read_csv('train/y.csv')

In [10]:
data_set = pd.get_dummies(data_set, columns=['TOOL','TOOL (#1)','TOOL (#2)','TOOL_ID (#1)','TOOL_ID (#2)','TOOL_ID (#3)','Tool','Tool (#1)','Tool (#2)',
                               'Tool (#3)','tool','tool (#1)'])

In [10]:
data_set = data_set.drop(['TOOL','TOOL (#1)','TOOL (#2)','TOOL_ID (#1)','TOOL_ID (#2)','TOOL_ID (#3)','Tool','Tool (#1)','Tool (#2)',
                               'Tool (#3)','tool','tool (#1)'], axis=1)

In [11]:
data_set

,ID,TOOL_ID,210X1,210X2,210X3,210X4,210X5,210X6,210X7,210X8,...,750X1443,750X1444,750X1445,750X1446,750X1447,750X1448,750X1449,750X1450,750X1451,750X1452
0,ID001,4.0,102.05,0.47,0.27,1.43,67.45,4.62,-0.54,-1.05,...,0.0,0.0,0.0,25.7,0.0,0.0,25.7,0.0,0.0,2.400000e+12
1,ID002,3.0,100.95,0.81,0.22,3.48,62.08,3.41,-2.12,1.02,...,0.0,0.0,0.0,25.5,0.0,0.0,25.5,0.0,0.0,2.400000e+12
2,ID003,2.0,98.56,0.56,0.24,1.17,56.70,3.08,-2.25,0.88,...,0.0,0.0,0.0,25.2,0.0,0.0,25.2,0.0,0.0,2.400000e+12
3,ID004,3.0,100.35,0.90,0.22,3.63,62.25,3.95,-1.98,0.82,...,0.0,0.0,0.0,26.4,0.0,0.0,26.4,0.0,0.0,2.400000e+12
4,ID005,3.0,100.25,0.85,0.23,3.43,61.42,3.63,-1.89,1.02,...,0.0,0.0,0.0,26.4,0.0,0.0,26.4,0.0,0.0,2.400000e+12
5,ID006,3.0,100.55,0.88,0.22,3.46,61.85,3.75,-2.10,0.93,...,0.0,0.0,0.0,26.5,0.0,0.0,26.5,0.0,0.0,2.400000e+12
6,ID007,4.0,102.20,0.44,0.27,1.43,67.38,4.63,-0.20,-0.44,...,0.0,0.0,0.0,26.5,0.0,0.0,26.5,0.0,0.0,2.400000e+12
7,ID010,4.0,102.25,0.42,0.27,1.31,67.47,4.61,-0.99,-0.99,...,0.0,0.0,0.0,25.5,0.0,0.0,25.5,0.0,0.0,2.400000e+12
8,ID011,4.0,102.25,0.42,0.27,1.31,67.44,4.63,0.32,-1.19,...,0.0,0.0,0.0,25.3,0.0,0.0,25.3,0.0,0.0,2.400000e+12
9,ID012,3.0,100.85,0.96,0.22,3.71,62.88,4.18,-2.14,0.99,...,0.0,0.0,0.0,25.2,0.0,0.0,25.2,0.0,0.0,2.400000e+12


In [12]:
data_set.drop(['ID', 'TOOL_ID'], axis=1, inplace=True)

In [13]:
data_set = drop_col(data_set)
data_set = quantile_dropout(data_set.ix[:, 1:])
data_set = drop_col(data_set)
data_set = data_set.transpose().drop_duplicates().transpose()
#data_set = date_cols(data_set)
#data_set = data_set.transpose().drop_duplicates().transpose()

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\pandas\core\generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [14]:
#data_set = data_set.transpose().drop_duplicates().transpose()
data_set

,210X2,210X3,210X4,210X5,210X6,210X7,210X8,210X9,210X10,210X11,...,750X1203,750X1225,750X1247,750X1288,750X1296,750X1298,750X1312,750X1314,750X1330,750X1384
0,0.47,0.27,1.43,67.450,4.62,-0.54,-1.05,-0.13,26.3,27.95,...,130.0,84.16,4.45,74256.0,11.6,0.77,36.8,4.53,4.47,49.9
1,0.81,0.22,1.31,62.080,3.41,-2.12,1.02,0.08,28.2,24.27,...,131.0,84.14,4.45,74323.0,11.6,0.76,36.8,4.53,4.47,50.0
2,0.56,0.24,1.17,56.700,3.08,-2.25,0.88,0.17,26.6,24.51,...,131.0,84.16,4.46,74453.0,11.6,0.76,36.8,4.53,4.47,50.0
3,0.51,0.22,1.31,62.250,3.95,-1.98,0.82,0.08,25.2,24.38,...,131.0,84.05,4.42,195952.0,11.6,0.76,36.8,4.54,4.48,50.0
4,0.85,0.23,1.31,61.420,3.63,-1.89,1.02,0.08,27.3,24.36,...,131.0,84.07,4.42,200350.0,11.6,0.76,36.8,4.53,4.48,50.0
5,0.88,0.22,1.31,61.850,3.75,-2.10,0.93,0.08,25.3,24.34,...,131.0,84.07,4.42,201542.0,11.6,0.76,36.8,4.54,4.49,49.9
6,0.44,0.27,1.43,67.380,4.63,-0.20,-0.44,-0.13,26.5,27.79,...,130.0,84.07,4.42,202110.0,11.6,0.76,36.8,4.53,4.49,49.9
7,0.42,0.27,1.31,67.470,4.61,-0.99,-0.99,-0.13,27.1,27.90,...,133.0,84.39,4.46,41829.0,11.6,0.76,36.7,4.53,4.47,50.0
8,0.42,0.27,1.31,67.440,4.63,0.32,-1.19,-0.14,25.9,27.88,...,134.0,84.39,4.46,42052.0,11.6,0.76,36.7,4.52,4.47,50.0
9,0.51,0.22,1.31,62.880,4.18,-2.14,0.99,0.08,27.7,24.37,...,134.0,84.39,4.46,42444.0,11.6,0.76,36.7,4.53,4.47,50.0


In [15]:
X = data_set.ix[:499, :]
X_test = data_set.ix[500:, :]

In [38]:
def wmaeEval(preds, dtrain):
    label = dtrain.get_label()
    return 'error', np.sum(np.square(preds - label)) / len(label)
param = {}
param['eta'] = 0.01
param['max_depth'] = 2

param['subsample'] = 0.9
param['colsample_bytree'] = 0.3
num_round = 3500

In [17]:
clf_gt = ensemble.GradientBoostingRegressor(max_depth=1, n_estimators=160)
clf_gt.fit(X, y)
# Set a minimum threshold of 0.25 
model = SelectFromModel(clf_gt, prefit=True) 
train = pd.DataFrame(model.transform(X))
test = pd.DataFrame(model.transform(X_test))


c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [39]:
xgbTrain = xgb.DMatrix(train, label=y)
modle = xgb.cv(param, xgbTrain, num_round, feval=wmaeEval, nfold=5)
print(modle.iloc[-1, 0])

0.022061


In [18]:
train

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,67.450,-1.05,27.95,0.53,450.0,53.14,4.47,753.700,0.22,5732.1,...,577.7,514.7,515.4,69541.0,16.2,15.2,4.080000,4.040000,9.87000,4.210000
1,62.080,1.02,24.27,0.52,430.0,63.70,4.43,946.430,0.20,5736.9,...,577.6,514.9,515.4,69542.0,14.3,14.4,4.090000,4.050000,9.87000,4.210000
2,56.700,0.88,24.51,0.52,425.0,46.92,6.05,599.010,0.38,5747.2,...,576.7,514.7,515.3,69543.0,14.6,15.2,4.140000,4.100000,9.88000,4.210000
3,62.250,0.82,24.38,0.52,430.0,61.87,5.24,874.520,0.21,5743.9,...,577.5,515.0,515.7,68971.0,14.2,15.0,3.940000,3.910000,9.91000,4.220000
4,61.420,1.02,24.36,0.52,430.0,61.41,5.13,1070.060,0.21,5741.6,...,578.0,514.9,515.8,68975.0,16.2,15.2,3.940000,3.910000,9.91000,4.220000
5,61.850,0.93,24.34,0.52,430.0,62.67,4.97,995.610,0.21,5742.4,...,578.3,515.0,515.8,68987.0,14.9,15.2,3.940000,3.910000,9.91000,4.240000
6,67.380,-0.44,27.79,0.50,450.0,53.48,4.69,766.790,0.21,5734.7,...,576.5,514.9,515.8,68992.0,14.9,15.2,3.930000,3.910000,9.91000,4.240000
7,67.470,-0.99,27.90,0.49,450.0,53.71,4.47,673.740,0.21,5734.4,...,576.2,515.1,515.9,68904.0,15.2,15.4,4.013718,3.970000,9.91000,4.240000
8,67.440,-1.19,27.88,0.49,450.0,53.65,4.49,796.660,0.21,5734.3,...,578.0,515.1,515.9,68907.0,15.2,15.4,4.013718,3.970000,9.90000,4.230000
9,62.880,0.99,24.37,0.52,430.0,63.84,4.96,865.350,0.21,5742.2,...,577.4,515.2,515.9,68911.0,14.5,15.0,4.013718,3.970000,9.89000,4.240000


In [71]:
sub = pd.read_csv('data/测试A-答案模板.csv', names=['ID'])
sub['res'] = result_xgb['res']
sub

,ID,res
0,ID716,2.691860
1,ID717,2.685901
2,ID719,2.910989
3,ID720,2.975187
4,ID721,2.861288
5,ID722,2.867144
6,ID723,2.849127
7,ID724,2.849484
8,ID725,2.765666
9,ID727,2.779486


In [72]:
sub.to_csv('submission{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, header=None)

In [147]:
lr = linear_model.LinearRegression()
cv_model = cross_val_score(lr, train1, y,  cv=10, scoring='neg_mean_squared_error')
np.mean(np.abs(cv_model))

0.02946633398280326

Stacking 线上

In [41]:
#5折交叉验证
x_train, x_valid, y_train, y_valid = train_test_split(train, y, test_size = 0.2)
x_train.reset_index(inplace=True)
x_train.drop(['index'], axis=1, inplace=True)

y_train.reset_index(inplace=True)
y_train.drop(['index'], axis=1, inplace=True)

y_valid.reset_index(inplace=True)
y_valid.drop(['index'], axis=1, inplace=True)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
model_rf = ensemble.RandomForestRegressor(n_estimators=750, max_depth=6)
model_gb = ensemble.GradientBoostingRegressor(n_estimators=2000, max_depth=1, subsample=0.9, learning_rate=0.01)
model_ar = ensemble.AdaBoostRegressor(learning_rate=0.01)
model_et = ensemble.ExtraTreesRegressor(max_depth=6, n_estimators=600)

lr = linear_model.LinearRegression()
sv = svm.SVR()
modle0  = xgb.XGBRegressor(learning_rate=0.01, max_depth=2, colsample_bytree=0.3, subsample=0.9, seed=0,  n_estimators=3000)
modle1  = xgb.XGBRegressor(learning_rate=0.01, max_depth=2, colsample_bytree=0.3, subsample=0.9, seed=1,  n_estimators=3300)
modle2  = xgb.XGBRegressor(learning_rate=0.01, max_depth=2, colsample_bytree=0.3, subsample=0.9, seed=2,  n_estimators=3300)
modle3  = xgb.XGBRegressor(learning_rate=0.01, max_depth=6, colsample_bytree=0.3, subsample=0.8, seed=3,  n_estimators=750)
clf0 = lgb.LGBMRegressor(colsample_bytree=0.3, learning_rate=0.01, subsample=0.9, num_leaves=4, objective='regression', n_estimators=3000, seed=0)
base_model = [['xgb1', modle0],
              ['xgb1', modle1],
#              ['xgb1', modle2],
              ['xgb1', model_gb],            
              ['lgb27', clf0],]

    
folds = list(KFold(len(x_train), n_folds=5, random_state=0))
S_train = np.zeros((x_train.shape[0], len(base_model)))
    #创建行数为测试样本数，列数为模型个数的矩阵
S_test = np.zeros((x_valid.shape[0], len(base_model)))    
for index, item in enumerate(base_model):
    print("the model", index)
    clf = item[1]
    S_test_i = np.zeros((x_valid.shape[0], len(folds)))
        #循环 对每一折验证训练模型
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = x_train.ix[train_idx, :]#训练集
        X_valid = x_train.ix[test_idx, :]#验证集 
        
        Y = y_train.ix[train_idx, :]
        clf.fit(X_train, Y['Y'])
        S_train[test_idx, index] = clf.predict(X_valid)
        S_test_i[:, j] = clf.predict(x_valid)       
    S_test[:, index] = S_test_i.mean(1)
    
print('Done')

the model 0
the model 1
the model 2
the model 3
Done


In [43]:
linreg = linear_model.Ridge(alpha=0.1)
linreg.fit(S_train, y_train)
result = linreg.predict(S_test)
evalerror(pd.DataFrame(result)[0],  y_valid['Y'])

0.019917743855673836

In [44]:
linreg = linear_model.LinearRegression()
linreg.fit(S_train, y_train)
result = linreg.predict(S_test)
evalerror(pd.DataFrame(result)[0],  y_valid['Y'])

0.020048221396421359

In [395]:
sub = pd.read_csv('data/测试A-答案模板.csv', names=['ID'])
sub['res'] = pd.DataFrame(result)[0]
sub

,ID,res
0,ID716,2.657439
1,ID717,2.693942
2,ID719,2.894279
3,ID720,2.955649
4,ID721,2.817205
5,ID722,2.840545
6,ID723,2.839927
7,ID724,2.838737
8,ID725,2.787729
9,ID727,2.760183


In [397]:
sub.to_csv('submission{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, header=None)